<a href="https://colab.research.google.com/github/Exw27/Blender-Rendering-With-Colab/blob/main/Blender_Render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### GPU Checker

In [1]:
# 1st cell
!nvidia-smi

Wed Jul 17 16:10:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

T4 GPU - Cuda


A100 GPU - Optix, Cuda


L4 GPU - Optix, Cuda


### Main

In [ ]:
!pip install gradio


In [6]:
import gradio as gr
from google.colab import drive
import os
import glob
from PIL import Image
import time

# Mount Google Drive
drive.mount('/content/drive')

# Ensure the directory for rendered images exists
rendered_images_dir = '/content/drive/MyDrive/BlenderRenderedImages'
os.makedirs(rendered_images_dir, exist_ok=True)

# Function to check GPU status
def check_gpu():
    gpu_status = os.popen('nvidia-smi').read()
    return gpu_status

# Function to download Blender
def download_blender():
    url = 'https://download.blender.org/release/Blender4.2/blender-4.2.0-linux-x64.tar.xz'
    os.system(f'wget {url}')
    os.system('tar xf blender-4.2.0-linux-x64.tar.xz')
    return 'Blender downloaded and unpacked.'

# Function to fix library issues
def fix_libraries():
    os.environ["LD_PRELOAD"] = ""
    os.system('apt remove -y libtcmalloc-minimal4')
    os.system('apt install -y libtcmalloc-minimal4')
    os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.0"
    return 'Libraries fixed.'

# Function to render a frame
def render_frame(filename, frame, device):
    output_path = os.path.join(rendered_images_dir, f"frame_{frame:04d}.png")
    command = f'./blender-4.2.0-linux-x64/blender -b {filename} -noaudio -E "CYCLES" -o "{output_path}" -f {frame} -F "PNG" -- --cycles-device {device}'

    start_time = time.time()
    os.system(command)
    elapsed_time = time.time() - start_time

    if os.path.exists(output_path):
        log = f'Frame {frame} rendered using {device}. Output saved to {output_path}. Render time: {elapsed_time:.2f} seconds.'
    else:
        log = f'Frame {frame} rendering failed. Command used: {command}'

    return log

# Function to render an animation
def render_animation(filename, start_frame, end_frame, device):
    output_path = os.path.join(rendered_images_dir, "frame_####.png")
    command = f'./blender-4.2.0-linux-x64/blender -b {filename} -noaudio -E "CYCLES" -o "{output_path}" -s {start_frame} -e {end_frame} -a -- --cycles-device {device}'

    start_time = time.time()
    os.system(command)
    elapsed_time = time.time() - start_time

    rendered_files = glob.glob(os.path.join(rendered_images_dir, f"frame_*.png"))
    if rendered_files:
        log = f'Animation frames {start_frame} to {end_frame} rendered using {device}. Outputs saved to {rendered_images_dir}. Render time: {elapsed_time:.2f} seconds.'
    else:
        log = f'Animation rendering failed. Command used: {command}'

    return log

# Function to display rendered images
def display_images():
    images = glob.glob(os.path.join(rendered_images_dir, '*.png'))
    images.sort()
    return images

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Blender Rendering with Google Colab")

    with gr.Tab("Setup"):
        gr.Markdown("### Step 1: Check GPU Status")
        gpu_output = gr.Textbox(label="GPU Status")
        gr.Button("Check GPU").click(check_gpu, outputs=gpu_output)

        gr.Markdown("### Step 2: Download Blender")
        download_output = gr.Textbox(label="Blender Download")
        gr.Button("Download Blender").click(download_blender, outputs=download_output)

        gr.Markdown("### Step 3: Fix Libraries")
        library_output = gr.Textbox(label="Library Fix")
        gr.Button("Fix Libraries").click(fix_libraries, outputs=library_output)

    with gr.Tab("Render"):
        gr.Markdown("### Render a Single Frame")
        blend_file = gr.Textbox(label="Blender File Path", value="/content/drive/MyDrive/Blender.blend")
        frame = gr.Number(label="Frame Number", value=1)
        device = gr.Dropdown(label="Device", choices=["CUDA", "OPTIX"], value="CUDA")
        render_output = gr.Textbox(label="Render Output")
        gr.Button("Render Frame").click(render_frame, inputs=[blend_file, frame, device], outputs=render_output)

        gr.Markdown("### Render an Animation")
        start_frame = gr.Number(label="Start Frame", value=1)
        end_frame = gr.Number(label="End Frame", value=250)
        render_anim_output = gr.Textbox(label="Render Animation Output")
        gr.Button("Render Animation").click(render_animation, inputs=[blend_file, start_frame, end_frame, device], outputs=render_anim_output)

    with gr.Tab("Rendered Images"):
        gr.Markdown("### View Rendered Images")
        image_gallery = gr.Gallery(label="Rendered Images")
        gr.Button("Refresh Images").click(display_images, outputs=image_gallery)

demo.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6f40ccafd568b20437.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
